<a href="https://colab.research.google.com/github/AndreVelasques7/Compilador/blob/main/ed-04/ed_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from dataclasses import dataclass
from typing import Callable
from random import randint, sample, random
from pprint import pprint
from typing import List, Tuple



@dataclass
class Cromossomo:
    """Representa um cromossomo no algoritmo genético."""
    dados: list[int]
    fitness: float = 0

@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int
    fitness: Callable[[Cromossomo], float]
    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]
    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]
    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float
    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]

# As funções torneio, crossover_1_corte, mutacao e elitismo do código original permanecem as mesmas.

def inicializar_populacao(tam_populacao: int, tam_cromossomo: int) -> list[Cromossomo]:
    """Inicializa uma população de cromossomos com genes aleatórios."""
    return [Cromossomo([randint(0, 1) for _ in range(tam_cromossomo)]) for _ in range(tam_populacao)]



def crossover_1_corte(casais: list[tuple[Cromossomo, Cromossomo]]) -> list[Cromossomo]:
    """Aplica o cruzamento de um ponto entre os pais selecionados."""
    filhos = []
    for pai1, pai2 in casais:
        corte = randint(1, len(pai1.dados) - 1)
        filhos.extend([
            Cromossomo(pai1.dados[:corte] + pai2.dados[corte:]),
            Cromossomo(pai2.dados[:corte] + pai1.dados[corte:])
        ])
    return filhos

def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    """Aplica mutação em um cromossomo com base em uma taxa de mutação."""
    for cromossomo in cromossomos:
        for i, gene in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                cromossomo.dados[i] = 1 - gene  # Inverte o gene (0 vira 1 e 1 vira 0)

def elitismo(populacao: list[Cromossomo]) -> list[Cromossomo]:
    """Seleciona a metade mais apta da população."""
    sorted_pop = sorted(populacao, key=lambda x: x.fitness, reverse=True)
    if sorted_pop:
        return sorted_pop[:len(sorted_pop)//2]
    else:
        return [populacao[0]]



def avaliar_solucao(cromossomo: Cromossomo) -> float:
    pilha_soma = [i+1 for i, gene in enumerate(cromossomo.dados) if gene == 1]
    pilha_produto = [i+1 for i, gene in enumerate(cromossomo.dados) if gene == 0]

    soma_total = sum(pilha_soma)
    produto_total = 1
    for card in pilha_produto:
        produto_total *= card

    penalidade_soma = abs(36 - soma_total)
    penalidade_produto = abs(360 - produto_total)

    return penalidade_soma + penalidade_produto

def algoritmo_genetico(config: Config, max_geracoes: int = 100) -> Tuple[List[int], List[int]]:
    t = 0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)

    while t < max_geracoes:
        for c in P:
            c.fitness = config.fitness(c)

        casais = config.selecionar_pais(P)

        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)

        for c in F:
            c.fitness = config.fitness(c)

        sobreviventes = config.selecionar_sobreviventes(P + F)
        if sobreviventes:
            P = sobreviventes
        else:
            P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)
        t += 1

    best_solution = min(P, key=lambda x: x.fitness)
    pilha_soma = [i+1 for i, gene in enumerate(best_solution.dados) if gene == 1]
    pilha_produto = [i+1 for i, gene in enumerate(best_solution.dados) if gene == 0]

    return pilha_soma, pilha_produto

# Ao chamar o algoritmo genético, especifique o número máximo de gerações:
pilha_soma, pilha_produto = algoritmo_genetico(config, max_geracoes=100)
print("Pilha Soma:", pilha_soma)
print("Pilha Produto:", pilha_produto)



def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    """Seleciona pais para cruzamento usando o método do torneio."""
    casais = []
    for _ in range(len(populacao) // 2):
        pai1 = max(sample(populacao, min(tam_torneio, len(populacao))), key=lambda x: x.fitness)
        pai2 = max(sample(populacao, min(tam_torneio, len(populacao))), key=lambda x: x.fitness)

        casais.append((pai1, pai2))
    return casais



config = Config(
    tam_cromossomo=10,
    tam_populacao=20,
    fitness=avaliar_solucao,
    taxa_mutacao=0.05,
    selecionar_pais=torneio,
    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)
pilha_soma, pilha_produto = algoritmo_genetico(config)
print("Pilha Soma:", pilha_soma)
print("Pilha Produto:", pilha_produto)


Pilha Soma: [2, 3, 6, 8, 9]
Pilha Produto: [1, 4, 5, 7, 10]
Pilha Soma: []
Pilha Produto: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
